In [ ]:
annualization_factor = 12 

def time_series_regression(portfolio, factors, FF3F=False, resid=False):
    # where ff3f = fama french 3 factor model 
    
    ff_report = pd.DataFrame(index=portfolio.columns)
    bm_residuals = pd.DataFrame(columns=portfolio.columns)

    x = sm.add_constant(factors)
    for portf in portfolio.columns:
        y = portfolio[portf]
        res = sm.OLS(y, x).fit()
        ff_report.loc[portf, 'alpha'] = res.params['const'] * annualization_factor
        ff_report.loc[portf, 'beta'] = res.params[1]
        
        if FF3F:
            ff_report.loc[portf, 'Size beta'] = res.params[2]
            ff_report.loc[portf, 'Value beta'] = res.params[3]

        ff_report.loc[portf, 'info_ratio'] = np.sqrt(annualization_factor) * res.params['const'] / res.resid.std()
        ff_report.loc[portf, 'treynor_ratio'] = annualization_factor * portfolio[portf].mean() / res.params[1]
        ff_report.loc[portf, 'R-squared'] = res.rsquared
        ff_report.loc[portf, 'Tracking Error'] = res.resid.std() * np.sqrt(annualization_factor)

        # Calculate the MAE
        mae = np.mean(np.abs(res.resid))
        ff_report.loc[portf, 'MAE'] = mae

        if resid:
            bm_residuals[portf] = res.resid
    if resid:
        return bm_residuals
    return ff_report

In [ ]:
def plot_cum_returns(df: pd.DataFrame): 
    (df+1).cumprod().plot()
    